##Setup


---




###Install packages, dependencies, and functions needed


In [ ]:
!apt-get update
!apt-get install git

# MNE-BIDS preprocessing functions
!pip install mne
!pip install MNE-bids
!pip install -r https://raw.githubusercontent.com/mne-tools/mne-bids-pipeline/main/requirements.txt
!git clone https://github.com/mne-tools/mne-bids-pipeline.git /cloned-mne-bids-pipeline

!pip install AutoReject
!pip install coffeine
!pip install Braindecode

!git clone https://github.com/meeg-ml-benchmarks/brain-age-benchmark-paper.git /cloned-paper-repo

!pip install -U matplotlib

!pip install -U scipy
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html


### Restart Runtime 

make sure all updated packages are loaded in their newest versions


In [ ]:
import torch
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

###Mount Google Drive for access to data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
######################################################################################################
#  ________   _________   ___    ___ ___  ___  ________  ________  ___  __      _______   _______
# |\   ___  \|\___   ___\|\  \  /  /|\  \|\  \|\   __  \|\   ____\|\  \|\  \   /  ___  \ /  ___  \
# \ \  \\ \  \|___ \  \_|\ \  \/  / | \  \\\  \ \  \|\  \ \  \___|\ \  \/  /|_/__/|_/  //__/|_/  /|
#  \ \  \\ \  \   \ \  \  \ \    / / \ \   __  \ \   __  \ \  \    \ \   ___  \__|//  / /__|//  / /
#   \ \  \\ \  \   \ \  \  /     \/   \ \  \ \  \ \  \ \  \ \  \____\ \  \\ \  \  /  /_/__  /  /_/__
#    \ \__\\ \__\   \ \__\/  /\   \    \ \__\ \__\ \__\ \__\ \_______\ \__\\ \__\|\________\\________\
#     \|__| \|__|    \|__/__/ /\ __\    \|__|\|__|\|__|\|__|\|_______|\|__| \|__| \|_______|\|_______|
#                        |__|/ \|__|
######################################################################################################

import mne
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge


# 250hz Autorejected data?


In [ ]:
import pickle
%cd /content/drive/My Drive/brainAge_competition/
with open(r"22chans_250hz_2secEpochs_autoreject.sav", "rb") as input_file:
  epoched_data = pickle.load(input_file)

# get the age to predict from the CSV file
train_path = "/content/drive/My Drive/brainAge_competition/raws/"
train_subj = 1200
meta = pd.read_csv(train_path + "train_subjects.csv")
y_train = []
for age in meta["age"][:train_subj]:
    y_train.append(age)
y_trainSet = y_train


# y's added to epoched data

In [ ]:
from numpy.matlib import repmat
from braindecode.datasets import WindowsDataset, BaseConcatDataset

for i in range(0,1200,1):
  print(i)
  current_y = np.matlib.repmat(y_trainSet[i],m=len(epoched_data[i]), n=1)
  metadata = np.array([
      list(range(len(epoched_data[i]))),  # i_window_in_trial (chunk of rec)
      len(epoched_data[i]) * [-1],  # i_start_in_trial (unknown / unused)
      len(epoched_data[i]) * [-1],  # i_stop_in_trial (unknown / unused
      current_y[:,0],  # target (e.g. subject age)
  ])
  metadata = pd.DataFrame(
      data=metadata.T,
      columns=['i_window_in_trial', 'i_start_in_trial', 'i_stop_in_trial',
                'target'],)
  epoched_data[i].metadata = metadata

for i in range(0,1200,1):
  print(epoched_data[i].metadata)

datasets = []
for i in range(0,1200,1):
  ds = WindowsDataset(
      windows=epoched_data[i],
      #description=description,
      targets_from='metadata',
      transform=None,
  )
  datasets.append(ds)

fullds = BaseConcatDataset(datasets)
fullds

# set-up analysis type + separate into train, valid, and test subsets

### this attempt built on code from Banville, H., Wood, S. U., Aimone, C., Engemann, D. A., & Gramfort, A. (2022). Robust learning from corrupted EEG with dynamic spatial filtering. NeuroImage, 251, 118994.

In [ ]:
import os
import argparse

import mne
import torch
from torch import nn
from braindecode import EEGClassifier
from braindecode.models import SleepStagerChambon2018
from braindecode.util import set_random_seeds
from skorch.helper import predefined_split
from skorch.callbacks import (
    Checkpoint, EarlyStopping, EpochScoring, LRScheduler)
from sklearn.metrics import (
    confusion_matrix, classification_report, balanced_accuracy_score)
from sklearn.utils.class_weight import compute_class_weight
import pickle

%cd /cloned-paper-repo
from deep_learning_utils import (
    create_dataset, create_model, create_estimator)

model_name = 'deep'
cropped = True
seed = 20211022


import os
%cd '/content/drive/MyDrive/dynamic_spatial_filtering/cloned-DSF-git'

save_dir="/content/drive/My Drive/brainAge_competition/DSFattempt2/"

denoising='no_denoising'
#denoising='autoreject'
#denoising='data_augm'

dsf_type = 'vanilla'
#dsf_type = 'dsfd'
#dsf_type = 'dsfm_st'

def get_exp_name(dsf_type, denoising):
    return f'{dsf_type}-{denoising}'

dir_name = get_exp_name(dsf_type, denoising)
import os
save_path = os.path.join(save_dir, dir_name)
if not os.path.exists(save_path):
    os.makedirs(save_path)

dsf_n_out_channels=None

!pip install -r requirements.txt


%cd '/content/drive/MyDrive/dynamic_spatial_filtering/cloned-DSF-git'
%ls
!pip install -r requirements.txt

from transforms import Compose, AdditiveWhiteNoise, logm_cov
from models import DynamicSpatialFilter
from utils import (
    load_data, apply_autoreject, split_dataset, none_or_int, get_exp_name,
    seed_np_rng)

seed = 20211022

if denoising == 'autoreject':
    windows_dataset = apply_autoreject(windows_dataset, seed, n_jobs)

# Split into train, valid and test sets
valid_size=0.2
test_size=0.2
random_state_valid=seed
random_state_test=seed

train_set, valid_set, test_set = split_dataset(
    fullds, valid_size, test_size,
    random_state_valid=random_state_valid,
    random_state_test=random_state_test)


# Add data transform if data_augm is being used

In [ ]:
import copy

import mne
import numpy as np
from autoreject import AutoReject
from autoreject.autoreject import _check_data, _apply_interp, _apply_drop
from sklearn.utils import check_random_state

class anthonyMod_AdditiveWhiteNoise(object):
    """Additive white noise.
    Parameters
    ----------
    p : float
        Probability that a channel receives white noise [0, 1].
    noise_strength : float | tuple
        Relative strength of the noise. The output of this transform is a
        convex combination of the original signal x and white noise n:
            y = (1 - w) * x + w * n
        If provided as a tuple (min_strength,max_strength), the relative
        strength will be uniformly sampled in the provided open interval.
    noise_std : float | tuple | None
        Standard deviation of the white noise. If provided as a tuple (min_std,
        max_std), the standard deviation will be uniformly sampled in the
        provided open interval. If None, the standard deviation will be the
        same as the standard deviation of the input signal.
    random_state : 'global' | np.random.RandomState | int | None
        Random state used to control noise parameters (channels to be
        corrupted, strength and standard deviation of noise). If 'global',
        random numbers will be generated with the `np.random` module so they
        use the global seed (this is useful to avoid duplicate augmentations
        when using transforms with num_workers > 1).
    noise_random_state : 'global' | np.random.RandomState | int | None
        Random state used to generate the white noise itself. If None, the
        random number generator will be initialized to the same as
        `random_state`'s. If 'global', random numbers will be generated with
        the `np.random` module so they use the global seed (this is useful to
        avoid duplicate augmentations when using transforms with
        num_workers > 1).
        NOTE: The two random states are kept separate so that it is possible to
              have identical recording-wise corruption on raw data and epoched
              data.
    recording_wise : bool
        If True and a 3D array (n_windows, n_channels, n_times) is passed to
        __call__, the same noise parameters will be used to generate the noise
        of all windows. If False, each window will be corupted with its own
        noise parameters.
    """
    __name__ = 'AdditiveWhiteNoise'

    def __init__(self, p, noise_strength, noise_std=None,
                 random_state='global', noise_random_state=None,
                 recording_wise=True):
        self.p = p
        self.noise_strength = noise_strength
        self.noise_std = noise_std
        self._set_random_states(random_state, noise_random_state)
        self.recording_wise = recording_wise
        print("Data Augmentation Is ON")   

    def _set_random_states(self, random_state, noise_random_state):
        if random_state == 'global':
            self.rng = np.random
        else:
            self.rng = check_random_state(random_state)

        if noise_random_state == 'global':
            self.noise_rng = np.random
        elif noise_random_state is None:
            if random_state == 'global':
                self.noise_rng = np.random
            else:
                self.noise_rng = copy.deepcopy(self.rng)
        else:
            self.noise_rng = check_random_state(noise_random_state)

    def __call__(self, X, mask=None):
        """Generate and apply white noise to an mne.Epochs object.
        Parameters
        ----------
        X : np.ndarray | mne.Epochs
            Data to be corrupted.
        mask : np.ndarray | None
            If provided as a numpy array of bool with shape (n_channels,), will
            replace the mask that is normally sampled at every call.
        Returns
        -------
        np.ndarray :
            Corrupted data.
        """
        #print("Data Augmentation Is ON")        
        X_out = X._data if isinstance(X, mne.Epochs) else X

        if X_out.ndim == 2:
            n_channels, n_times = X_out.shape
        elif X_out.ndim == 3:
            if not self.recording_wise:
                all_Xi = [self.__call__(Xi, mask) for Xi in X_out]
                return np.stack(all_Xi, axis=0)
            n_windows, n_channels, n_times = X_out.shape
        else:
            raise ValueError(
                f'Data must have 2 or 3 dimensions, got {X_out.ndim}')

        # Pick channels
        if mask is None:
            mask = self.rng.binomial(1, self.p, n_channels) == 1
        n_bad_chs = sum(mask)

        if n_bad_chs > 0:
            if isinstance(self.noise_std, (int, float)):
                loc = 0
                scale = self.noise_std
            elif isinstance(self.noise_std, tuple):
                loc = np.zeros(n_bad_chs)
                scale = self.rng.uniform(
                    low=self.noise_std[0], high=self.noise_std[1],
                    size=n_bad_chs)
            elif self.noise_std is None:
                loc = np.zeros(n_bad_chs)
                scale = X[mask].std(axis=1)
            else:
                raise ValueError(
                    'noise_std must be an int, float, tuple or None, got '
                    f'{type(self.noise_std)}.')

            if X_out.ndim == 2:
                n = self.noise_rng.normal(
                    loc=loc, scale=scale, size=(n_times, n_bad_chs)).T
            elif X_out.ndim == 3:
                n = self.noise_rng.normal(
                    loc=loc, scale=scale, size=(n_windows, n_times, n_bad_chs))
                n = np.transpose(n, (0, 2, 1))

            if isinstance(self.noise_strength, tuple):
                w = self.rng.uniform(
                    low=self.noise_strength[0], high=self.noise_strength[1])
            else:
                w = self.noise_strength

            if X_out.ndim == 2:
                X_out[mask] = (1 - w) * X_out[mask] + w * n
            elif X_out.ndim == 3:
                X_out[:, mask] = (1 - w) * X_out[:, mask] + w * n

        return X_out

if denoising == 'data_augm':
    train_set.transform = anthonyMod_AdditiveWhiteNoise(
        p=0.5, noise_strength=(0.5, 1), noise_std=(20, 50),
        recording_wise=False)
    
#train_set.transform


# Continued set-up

In [ ]:


n_epochs = 40
batch_size = 128 #128 #if dataset == 'camcan' else 256
cropped = True

import torch
cuda = torch.cuda.is_available() 

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False # not sure / dsf paper had this as false I think / #not args.deterministic
torch.backends.cudnn.deterministic = True # args.deterministic
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
set_random_seeds(seed=seed, cuda=cuda)

################################################################################################
# age regression pipleine code

from deep_learning_utils import ( create_model,
    create_estimator)

x, y, _ = train_set[0]

n_channels, window_size = x.shape

y
n_channels
window_size


x

array([[ 9.9367018e-05,  9.3538067e-05,  8.9370544e-05, ...,
         4.0873133e-06,  1.9670015e-06,  4.7957252e-07],
       [ 8.8532397e-05,  8.5109525e-05,  8.1272403e-05, ...,
         1.4493757e-05,  1.4379733e-05,  1.2860188e-05],
       [ 1.0450727e-04,  9.9104902e-05,  9.5193347e-05, ...,
         1.0827463e-05,  7.6711913e-06,  5.2751857e-06],
       ...,
       [-1.4357081e-05, -1.6948701e-05, -1.9407458e-05, ...,
        -3.9371651e-05, -4.0957515e-05, -3.9101462e-05],
       [ 2.3919374e-05,  1.7511162e-05,  1.4188075e-05, ...,
        -3.0115942e-05, -3.1787542e-05, -3.1874457e-05],
       [ 5.8286274e-05,  5.2283671e-05,  4.7506655e-05, ...,
        -1.7269940e-05, -1.8794333e-05, -1.9226192e-05]], dtype=float32)

# Create model

In [ ]:

###############################################################################################
# our output has 3 dimensions, probably b/c we didn't do the X = slice dataset step

#def anthonyMod_squeeze_to_ch_x_classes(x):
#    """Squeeze the model output from any dimension to batch_size x n_classes."""
#    while x.size()[-1] == 1 and x.ndim > 2:
#        print(x.shape)
#        x = x.squeeze(x.ndim-1)
#    return x

def anthonyMod_squeeze_to_ch_x_classes(x):
    """Squeeze the model output from any dimension to batch_size x n_classes."""
    while x.size()[-1] == 1 and x.ndim >= 2:
        x = x.squeeze(x.ndim-1)
    return x

import mne
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from joblib.parallel import Parallel, delayed

from mne_bids import BIDSPath

from skorch.callbacks import LRScheduler, BatchScoring
from skorch.helper import SliceDataset

from braindecode.datasets import WindowsDataset, BaseConcatDataset
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, Deep4Net
from braindecode.models.util import to_dense_prediction_model
from braindecode.models.modules import Expression
from braindecode import EEGRegressor

def create_model_v2(model_name, window_size, n_channels, cropped, seed):
    """Create a braindecode model (either ShallowFBCSPNet or Deep4Net).
    Parameters
    ----------
    model_name: str
        The name of the model (either 'shallow' or 'deep').
    window_size: int
        The length of the input data time series in samples.
    n_channels: int
        The number of input data channels.
    cropped: bool
        A flag to switch between cropped and trialwise decoding.
    seed: int
        The seed to be used to initialize the network.
    Returns
    -------
    model: braindecode.models.Deep4Net or braindecode.models.ShallowFBCSPNet
        A braindecode convolutional neural network.
    lr: float
        The learning rate to be used in network training.
    weight_decay: float
        The weight decay to be used in network training.
    """
    # check if GPU is available, if True chooses to use it
    cuda = torch.cuda.is_available()
    if cuda:
        torch.backends.cudnn.benchmark = True
    # Set random seed to be able to reproduce results
    set_random_seeds(seed=seed, cuda=cuda)

    final_conv_length = 'auto'
    if model_name == 'shallow':
        if cropped:
            window_size = None
            final_conv_length = 35
        model = ShallowFBCSPNet(
            in_chans=n_channels,
            n_classes=1,
            input_window_samples=window_size,
            final_conv_length=final_conv_length,
        )
        lr = 0.0625 * 0.01 
        weight_decay = 0
    elif model_name == 'deep':
        if cropped:
            window_size = None
            final_conv_length = 1
        model = Deep4Net(
            in_chans=n_channels,
            n_classes=1,
            input_window_samples=window_size,
            final_conv_length=final_conv_length,
            stride_before_pool=True,
        )
        lr = 1 * 0.01
        weight_decay = 0.5 * 0.001
    else:
        raise ValueError(f'Model {model_name} unknown.')

    if cropped:
        to_dense_prediction_model(model)

    # remove the softmax layer from models
    new_model = torch.nn.Sequential()
    for name, module_ in model.named_children():
        if "softmax" in name:
            continue
        new_model.add_module(name, module_)

    if cropped:
        new_model.add_module('global_pool', torch.nn.AdaptiveAvgPool1d(1))
        new_model.add_module('squeeze2', Expression(anthonyMod_squeeze_to_ch_x_classes))
    model = new_model

    # Send model to GPU
    if cuda:
        model.cuda()
        n_devices = torch.cuda.device_count()
        if n_devices > 1:
            print(f'Using {n_devices} GPUs.')
            model = nn.DataParallel(model)

    return model, lr, weight_decay


################################# dsf pt 1/2 #############################################

import torch
from torch import nn

def soft_thresholding(x, b, a=None):
    """Remap values between [-a, b] to 0, keep the rest linear.
    """
    if a is None:
        a = b
    return (torch.clamp(x - b, min=0) * (x > 0) +
            torch.clamp(x + a, max=0) * (x <= 0))

def logm_eig(A, spd=True):
    """Batched matrix logarithm through eigenvalue decomposition.

    Parameters
    ----------
    A : torch.Tensor
        Square matrices of shape (B, F, C, T).

    Returns
    -------
    torch.Tensor :
        Matrix logarithm of A.
    """
    e, v = torch.symeig(A, eigenvectors=True)
    e = torch.clamp(e, min=1e-10)  # clamp the eigenvalues to avoid -inf
    return v @ torch.diag_embed(
        torch.log(e), dim1=2, dim2=3) @ v.transpose(2, 3)

class SpatialFeatureExtractor(nn.Module):
    """Extract spatial features from input.
    """
    def __init__(self, kind, n_channels):
        super().__init__()
        self.kind = kind
        self.n_channels = n_channels
        self.inds = torch.triu_indices(n_channels, n_channels)

    @staticmethod
    def _cov(x):
        xm = x - x.mean(axis=3, keepdims=True)
        return xm @ xm.transpose(2, 3) / (x.shape[3] - 1)

    def forward(self, x):
        """
        x.shape = (B, F, C, T)
        """
        if self.kind == 'log_diag_cov':
            out = torch.log(torch.var(x, 3, unbiased=True))
            out[torch.isneginf(out)] = 0
        elif self.kind == 'logm_cov_eig':
            cov = self._cov(x)
            logm_cov = logm_eig(cov)
            out = logm_cov[:, :, self.inds[0], self.inds[1]]
        else:
            out = None

        return out

    @property
    def n_outputs(self):
        if self.kind == 'log_diag_cov':
            return self.n_channels
        else:
            return int(self.n_channels * (self.n_channels + 1) / 2)

class AnthonyMod_DynamicSpatialFilter(nn.Module):
    """Dynamic spatial filter module.

    Input: (B, F, C, T) [F is the number of filters]
    Output: (B, F, C', T) [transformed input]

    Parameters
    ----------
    n_channels : int
        Number of input channel.
    mlp_input : str
        What to feed the MLP. See SpatialFeatureExtractor.
    n_hidden : int | None
        Number of hidden neurons in the MLP. If None, use `ratio`.
    ratio : float
        If `n_hidden` is None, the number of hidden neurons in the MLP is
        computed as int(ratio * n_inputs).
    n_out_channels : int | None
        Number of output ("virtual") channels in the DSF-based models (only
        affects DSF models). If None, n_out_channels = n_channels.
    apply_soft_thresholding : bool
        If True, apply soft thresholding to the spatial filter matrix W.
    return_att : bool
        If True, `forward()` returns attention values as well. Used for
        inspecting the model.
    """
    def __init__(self, n_channels, mlp_input='log_diag_cov', n_hidden=None,
                 ratio=1, n_out_channels=None, apply_soft_thresh=False,
                 return_att=False):
        super().__init__()
        self.apply_soft_thresh = apply_soft_thresh
        self.return_att = return_att

        # Initialize spatial feature extractor
        self.feat_extractor = SpatialFeatureExtractor(
            mlp_input, n_channels)
        n_inputs = self.feat_extractor.n_outputs
        if n_hidden is None:
            n_hidden = int(ratio * n_inputs)

        # Define number of outputs
        if n_out_channels is None:
            n_out_channels = n_channels
        self.n_out_channels = n_out_channels
        n_outputs = n_out_channels * (n_channels + 1)

        self.mlp = nn.Sequential(
            nn.Linear(n_inputs, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_outputs)
        )
        print("DSF is ON")

    def forward(self, x):
        if isinstance(x, list):  # logm was computed on CPU with transforms
            x, feats = x
            feats = feats.unsqueeze(1)
        else:
            feats = None
        if x.ndim == 3:
            b, c, _ = x.shape
            f = 1
            x = x.unsqueeze(1)
        elif x.ndim == 4:
            b, f, c, _ = x.shape

        mlp_out = self.mlp(self.feat_extractor(x) if feats is None else feats)

        W = mlp_out[:, :, self.n_out_channels:].view(
            b, f, self.n_out_channels, c)
        if self.apply_soft_thresh:
            W = soft_thresholding(W, 0.1)
        bias = mlp_out[:, :, :self.n_out_channels].view(
            b, f, self.n_out_channels, 1)
        out = W @ x + bias
        out = torch.squeeze(out,1) # squeeze second dim
        if self.return_att:
            return out, (W, bias)
        else:
            return out

##########################

if dsf_type != 'vanilla':
    if dsf_type == 'dsfd':
        mlp_input = 'log_diag_cov'
        dsf_soft_thresh = False
    elif dsf_type == 'dsfm_st':
        mlp_input = 'logm_cov_eig'
        dsf_soft_thresh = True

        # Use CPU to compute logm, it's faster than pytorch with cuda
        train_set.transform = logm_cov if train_set.transform[0] is None \
            else Compose([train_set.transform[0], logm_cov])
        valid_set.transform = logm_cov
        test_set.transform = logm_cov

    else:
        raise ValueError(
            f'dsf_type must be None, dsfd or dsfm_st, got {dsf_type}')
    dsf = AnthonyMod_DynamicSpatialFilter(
        n_channels, mlp_input=mlp_input,
        n_out_channels=dsf_n_out_channels,
        apply_soft_thresh=dsf_soft_thresh)
    n_channels = dsf.n_out_channels




##################################################################################


model_name='deep'

model, lr, weight_decay = create_model_v2(
    model_name=model_name,
    window_size=window_size,
    n_channels=n_channels,
    cropped=True,
    seed=seed,
)

model

################################# dsf pt 2/2 #############################################

if dsf_type != 'vanilla':
    model = nn.Sequential(dsf, model)
  
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nModel has {n_params} trainable parameters.\n')

if torch.cuda.device_count() > 1:  # Parallelize model over GPUs
    print(f'\nUsing {torch.cuda.device_count()} GPUs.\n')
    model = nn.DataParallel(model)

##################################################################################
print(lr)

# Wrap model in estimator + more set-up

In [ ]:
n_jobs=1
from skorch.callbacks import LRScheduler, BatchScoring

# incorporate checkpoint so we can get the optimal params
from skorch.callbacks import (
    Checkpoint, EarlyStopping, EpochScoring, LRScheduler)

num_workers=1

cp = Checkpoint(dirname=save_path)
#patience= 7 # with such small ptps let's use full epochs  # dsf paper used 7 # default is 5
#early_stopping = EarlyStopping(patience=patience)

callbacks = [
    # can be dropped if there is no interest in progress of _window_ r2
    # during training
    ("R2", BatchScoring('r2', lower_is_better=False, on_train=True)),
    # can be dropped if there is no interest in progress of _window_ mae
    # during training
    ("MAE", BatchScoring("neg_mean_absolute_error",
                          lower_is_better=False, on_train=True)),
    ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ("cp",cp),
    #('patience', early_stopping)
]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    # Too many workers can create an IO bottleneck - n_gpus * 5 is a good
    # rule of thumb
    max_num_workers = torch.cuda.device_count() * 5
    num_workers = min(max_num_workers, n_jobs if n_jobs > 1 else 0)
else:
    num_workers = n_jobs if n_jobs > 1 else 0

from braindecode import EEGRegressor

model.pool_4.dilation = (31,1) # since using 2 sec epochs instead of 10 sec epochs

estimator = EEGRegressor(
    model,
    criterion=torch.nn.L1Loss,  # optimize MAE
    optimizer=torch.optim.AdamW,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    max_epochs=n_epochs,
    train_split=predefined_split(valid_set), #train_split=None,  # we do splitting via KFold object in cross_validate
    batch_size=batch_size,
    callbacks=callbacks,
    device=device,
    iterator_train__num_workers=num_workers,
    iterator_valid__num_workers=num_workers,
    iterator_train__shuffle=True,
    iterator_train__worker_init_fn=seed_np_rng
)


estimator



# Fit and evaluate model

In [ ]:

mne.set_log_level('ERROR')
estimator
estimator.fit(train_set, y=None)

# need to modify neural net since we using 2 sec data instead of 10 sec data

#    return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)\
# Given input size: (200x63x1). Calculated output size: (200x-99x1). Output size is too small

# i think it's at pool_4
# should we just go to 10 sec @ 200hz for 2001 samples per epoch to avoid this for now? 
# pool_4 dialtion from 81,1 to 21,1 and yup running, 31,1 got better performance on quick comparison so using

import gc
gc.collect()
torch.cuda.empty_cache()

# Load best model
estimator.initialize()
estimator.load_params(checkpoint=cp)
save_path = '/content/drive/My Drive/brainAge_competition/DSFattempt2/vanilla-no_denoising'


# Pickle best model
with open(os.path.join(save_path, 'best_model.pkl'), 'wb') as f:
    estimator.train_split = None  # Avoid pickling the validation set
    pickle.dump(estimator, f)

#%% 5- Evaluate performance
len(test_set)
test_set[0][1]

y_true_test=[]
for i in range(0,len(test_set),1):
  y_true_test.append(test_set[i][1])

y_true_test = np.array(y_true_test)
y_true_test.shape

#y_true_test = test_set.get_metadata()['target'].to_numpy()
y_pred_test = estimator.predict(test_set)

x=y_pred_test
y=y_true_test

from sklearn.metrics import r2_score, mean_absolute_error
MAE=mean_absolute_error(y_true_test, y_pred_test)
rsq=r2_score(y_true_test, y_pred_test)

print(f'MAE = {MAE} and r^2 = {rsq}')